In [20]:
import torch
import clip
import os

In [19]:
fpath = "/ckb-nfs/home/zcafego/git/CoOp/output/imagenet/CoOp/vit_l14_ep50_1shots/nctx16_cscFalse_ctpend/seed3/prompt_learner/model.pth.tar-50"

In [24]:
clip_model, clip_preprocessor = clip.load('ViT-L/14', device='cpu', download_root=os.path.expanduser('~/.cache/clip'))

In [3]:
prompt_learner = torch.load(fpath, map_location='cpu')

In [30]:
state_dict = prompt_learner['state_dict']
context = state_dict['ctx']
del state_dict['token_prefix']
del state_dict['token_suffix']

In [31]:
state_dict

OrderedDict([('ctx',
              tensor([[-0.0715,  0.0522, -0.0568,  ..., -0.0718,  0.0073,  0.0138],
                      [ 0.0116,  0.0768,  0.0134,  ...,  0.0413, -0.0926, -0.0156],
                      [-0.0660,  0.0338, -0.0153,  ..., -0.0483, -0.0496,  0.0276],
                      ...,
                      [-0.0456, -0.0377, -0.0662,  ..., -0.0372, -0.0216, -0.0146],
                      [-0.0619, -0.0449,  0.0169,  ..., -0.0164, -0.0257, -0.0486],
                      [-0.0397,  0.0165,  0.0873,  ...,  0.0305, -0.0193, -0.0122]],
                     dtype=torch.float16))])

In [25]:
transformer = clip_model.transformer
positional_embedding = clip_model.positional_embedding
ln_final = clip_model.ln_final
text_projection = clip_model.text_projection
dtype = clip_model.dtype

In [28]:
positional_embedding

Parameter containing:
tensor([[ 0.0016,  0.0020,  0.0002,  ..., -0.0013,  0.0008,  0.0015],
        [ 0.0042,  0.0029,  0.0002,  ...,  0.0010,  0.0015, -0.0012],
        [ 0.0018,  0.0007, -0.0012,  ..., -0.0029, -0.0009,  0.0026],
        ...,
        [ 0.0216,  0.0055, -0.0101,  ..., -0.0065, -0.0029,  0.0037],
        [ 0.0188,  0.0073, -0.0077,  ..., -0.0025, -0.0009,  0.0057],
        [ 0.0330,  0.0281,  0.0289,  ...,  0.0160,  0.0102, -0.0310]],
       requires_grad=True)

In [ ]:
x = prompts + positional_embedding.type(dtype)
x = x.permute(1, 0, 2)  # NLD -> LND
x = transformer(x)
x = x.permute(1, 0, 2)  # LND -> NLD
x = ln_final(x).type(dtype)

# x.shape = [batch_size, n_ctx, transformer.width]
# take features from the eot embedding (eot_token is the highest number in each sequence)
x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection